<a href="https://colab.research.google.com/github/josephassiga/tensorflow-and-keras-tutorial/blob/main/W3/Lab5/W3_Lab_1_intro_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing (NLP)

NLP is about: using machine learning and large datasets to
give computers the ability not to understand language, which is a more lofty goal, but to ingest a piece of language as input and return something useful, like predicting the following:

*  What’s the topic of this text?: text classification.
*  Does this text contain abuse?: content filtering.
*  Does this text sound positive or negative?: sentiment analysis.
*  What should be the next word in this incomplete sentence?: language modeling
*  How would you say this in German?: translation.
*  How would you summarize this article in one paragraph?” : summarization.





# Preparing text data

Deep learning models, being differentiable functions, can only process numeric tensors: they can’t take raw text as input. Vectorizing text is the process of transforming text into numeric tensors :

* First, you standardize the text to make it easier to process, such as by converting it to lowercase or removing punctuation.
* You split the text into units (called tokens), such as characters, words, or groups of words. This is called tokenization.
* You convert each such token into a numerical vector. This will usually involve
first indexing all tokens present in the data.

Let's review each of these steps:

![From raw text to vectors](images/text-vectorization.png)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

Now inspecting the data of the Jena weather dataset